In [13]:
import pandas as pd
import glob
import re
import collections
import numpy as np
import matplotlib.pyplot as plt
import gzip
import datetime
import pickle
import itertools

#from tqdm.notebook import tqdm
from tqdm import tqdm
from nltk.util import ngrams
from wordcloud import WordCloud
from IPython.display import display, HTML

import spacy
from spacy import displacy
from collections import Counter
from bisect import bisect_left

nlp = spacy.load('en_core_web_sm')

path_to_folder = '/Users/rwkoops/PycharmProjects/IR_project/IR_project_02/'

In [2]:
# Debug
print(datetime.datetime.now().time())

21:08:39.366110


# Create featureset
Load the dataset with the generated synthetic candidate queries

In [3]:
syn_candidate_queries = pd.read_pickle(path_to_folder + 'created_sample/syn_candidate_queries_1m.pickle')

# Create N-gram & NER features

#### Load it

In [4]:
with open(path_to_folder + 'created_sample/hist_suffixes.pickle_1m', 'rb') as f:
    suffixes = pickle.load(f)

#### Create N-gram features based on the "Query_clean"-column

In [5]:
historical_dict = collections.Counter(suffixes)

def ngram_freq_per_n(candidate, historical_dict, n):
    words = candidate.split()
    ngram_n = 0
    ngrams_i = ngrams(words, n)
    
    for word in ngrams_i:
        freq_g = historical_dict[" ".join(word)]
        ngram_n += freq_g
        
    return ngram_n

In [15]:
ngram_max = 6

tqdm.pandas()  # <- added this for timer

for i in range(1, ngram_max + 1):
    ngram_name = 'ngram_' + str(i)
    syn_candidate_queries[ngram_name] = syn_candidate_queries[syn_candidate_queries.notnull()]['Query_clean'].progress_map(lambda candidate_row: ngram_freq_per_n(candidate_row, historical_dict, i))

/usr/local/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 1093771/1093771 [00:04<00:00, 248155.01it/s]


#### Create NER Features based on the "Synthetic_query"-column

In [16]:
def ner_contains_and_count_norm(candidate): 
    entities = nlp(str(candidate))
    contains = 0
    ner_norm = 0
    
    if len(entities) > 0:
        contains = 1
        ner_norm = len(entities) / len(candidate)
    
    return [contains, ner_norm]

tqdm.pandas()  # <- added this for timer

syn_candidate_queries['has_ne'], syn_candidate_queries['ne_norm'] = zip(*syn_candidate_queries[syn_candidate_queries.notnull()]['Synthetic_query'].progress_map(lambda candidate_row: ner_contains_and_count_norm(candidate_row)))

100%|██████████| 1093771/1093771 [1:48:24<00:00, 168.15it/s] 


#### Save intermediate syn_candidate_queries with pickle

In [21]:
syn_candidate_queries.to_pickle(path_to_folder + 'created_sample/feature_vector_rough_1m.pickle')

#### Save feature_vector as CSV-file

In [22]:
feature_vector.to_csv(path_to_folder + 'created_sample/feature_vector_rough_1m.csv', index = False)

#### Load intermediate syn_candidate_queries as feature_vector with pickle

In [23]:
feature_vector = pd.read_pickle(path_to_folder + 'created_sample/feature_vector_rough_1m.pickle')

# Create 'Other'-features

In [19]:
def get_other_features(prefix, suffix, candidate, historical_logs):
    # Boolean indicating whether the prefix ends with a space
    bool_space = prefix.endswith(" ")
    
    # The frequency of the candidate in the historical logs
    frequency = historical_logs[candidate]
    
    # Prefix, suffix and total length in characters
    prefixlen_char = len(prefix)
    suffixlen_char = len(suffix)
    totallen_char = len(candidate)
    
    # Prefix, suffix and total length in words
    prefixlen_word = len(prefix.split())
    suffixlen_word = len(suffix.split())
    totallen_word = len(candidate.split())
    
    return [frequency, 
            prefixlen_char, suffixlen_char, totallen_char,
            prefixlen_word, suffixlen_word, totallen_word,
            bool_space * 1]

In [24]:
feature_vector['candid_freq'], feature_vector['prefixlen_char'], feature_vector['suffixlen_char'], feature_vector['totallen_char'], feature_vector['prefixlen_word'], feature_vector['suffixlen_word'], feature_vector['totallen_word'], feature_vector['bool_space'] = zip(*feature_vector.apply(lambda query_row: get_other_features(query_row.Prefix, query_row.Suffix, query_row.Synthetic_query, historical_dict), axis=1))

# Create "BERT"-feature

In [ ]:
# from __future__ import print_function

# import mxnet as mx
# import mxnet.ndarray as nd
# from mxnet import nd, autograd, gluon
# from mxnet.gluon.data.vision import transforms

# import warnings


In [ ]:
# def get_BERT(prefix, suffix, bert_model):
#     with warnings.catch_warnings():
#         warnings.simplefilter("ignore")
#         deserialized_net = gluon.nn.SymbolBlock.imports(path_to_folder + 'created_sample/trained_bert_classifier_10k-symbol.json', ['data'], path_to_folder + 'created_sample/trained_bert_classifier_10k-0003.params', ctx=ctx)


#         verify_loaded_model(deserialized_net)

#### Save feature_vector as CSV-file

In [25]:
feature_vector.to_csv(path_to_folder + 'created_sample/feature_vector_1m.csv', index = False)

#### Save feature_vector as pickle file

In [26]:
feature_vector.to_pickle(path_to_folder + 'created_sample/feature_vector_1m.pickle')

# Cleaned feature_vector

In [29]:
cl_feat_vec = feature_vector

del cl_feat_vec['Query_clean']
del cl_feat_vec['Prefix']
del cl_feat_vec['Suffix']
del cl_feat_vec['Synthetic_query']

cl_feat_vec.to_csv(path_to_folder + 'created_sample/feature_vec_opt_1m.csv', index = False)
cl_feat_vec.to_pickle(path_to_folder + 'created_sample/feature_vec_opt_1m.pickle')

In [27]:
# Debug
print(datetime.datetime.now().time())

23:09:06.511441
